# Modelo definitivo

CHONA ME DIJO QUE USARA LOS DATOS SIN PCA Y QUE HAGA UN POOLING GRANDE AL PRINCIPIO DE LA CNN PARA ACHICAR LOS DATOS DE UNA CUANDO ENTRAN. 
ADEMÁS, TENGO QUE HACER CONV2D EN VES DE 1D Y TENGO QUE AGREGARLE LOS NUEVOS DATOS. ADEMÁS TENGO QUE USAR EL PRETRAINED MODEL RESNET QUE USO 
EL INDIO EN EL VIDEO PARA TENER YO YA HECHO EL COSO Y CON LOS WEIGHTS Y QUE NO ME TRADE TANTO EL ENTRENAMIENTO. YO LE CAMBIARÍA LO DEL POOL AL 
PRINCIPIO Y LO ADAPTARÍA AL FINAL DE LA RED PARA QUE SE AJUSTE A MIS DATOS.
- TENGO QUE CORRER EL CODIGO PARA EL DATASET DE NUEVO PERO SIN EL PCA 
- VOLVER A ARREGLAR LA CNN PARA QUE ADMITA LA VIEJA INPUT SHAPE
- USAR UN MODELO RESNET PREENTRENADO CON DATOS COHERENTES (PREFERENTEMENTE EL QUE USÓ EL INDIO)
- INCORPORAR LO QUE ME DIJO CHONA DE CONV2D Y UN POOLING PARA QUE SE ADAPTE A RESNET (AdaptiveAvgPool2d - https://stackoverflow.com/questions/61955991/change-input-shape-dimensions-for-resnet-model-pytorch)

CODIGOS DEL INDIO:
https://github.com/talhaanwarch/youtube-tutorials/blob/main/eeg-conv2d.ipynb
https://github.com/talhaanwarch/youtube-tutorials/blob/main/eeg_epilepsy.ipynb

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D, Dropout, Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import tensorflow as tf
#import tensorflowjs
import pandas as pd
import numpy as np
import os
import gc
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
'''import wandb
import random
wandb.login(key="3750a6378bb91e331366851db2db2f8b674797f6")'''

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\47575909/.netrc


True

In [22]:
'''# Inicializa una ejecución de WandB
wandb.init(project="SinapsIA", entity="schajrisgaratiagustina")

# Configura hiperparámetros
config = wandb.config
config.learning_rate = 0.001
config.batch_size = 64'''

Problem at: C:\Users\47575909\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_init.py 848 getcaller


KeyboardInterrupt: 

In [2]:
#X = np.load('C:\\Users\\47575909\\Desktop\\X_12.npy')
#y = np.load('C:\\Users\\47575909\\Desktop\\y_12.npy')


In [3]:
#X.shape, y.shape

((1790, 224, 224, 3), (1790,))

In [4]:
# X = X.reshape(1790, 244, 244, 3)

In [5]:
# X = X[:,:224,:224,:]

In [6]:
#X.shape

(1790, 224, 224, 3)

In [10]:
input_tensor = Input(shape=(224, 224, 3))

#Cargo el modelo ResNet50 preentrenado
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)

#Congelo las capas del modelo base para que no se actualicen en el entrenamiento
for layer in base_model.layers:
    layer.trainable = False

#Agrego GlobalAveragePooling2D para adaptar la salida del modelo base
#x = GlobalAveragePooling2D()(base_model.output)

#Agrego capas adicionales para la clasificación de mis datos
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(500, activation='relu')(x)  # Puedes usar Dense en lugar de Conv2D aquí
x = Dense(200, activation='relu')(x)  # Capas Dense en lugar de MaxPooling2D
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

'''#x = base_model.output
#x = GlobalAveragePooling2D()(x)
x = Conv2D(filters=16, kernel_size=3, activation='relu')(base_model.output)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=16, kernel_size=3, activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Flatten()(x)
x = Dense(8, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='relu')(x)

output = Dense(1, activation='softmax')(x)'''

#Junto todo y creo el modelo
model = Model(inputs=input_tensor, outputs=output)

In [11]:
#COMPILO
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
'''def data_generator():
    for i in range(10):
        X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_{i+1}.npy')
        y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_{i+1}.npy')
        yield X, y

# https://stackoverflow.com/a/39271995'''

In [12]:
for i in range(2):
    X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_{i+1}.npy')
    y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_{i+1}.npy')

    #DIVIDO EN TRAIN, TEST Y EVALUATION
    #Divido los datos de train de los de test y evaluation (70/30)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

    #para parar el entrenamiento cuando llega a 90
    #early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

    #EVALÚO
    eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
    del X
    del y
    del X_val
    del y_val
    del X_train
    del y_train

    gc.collect()

    print("Accuracy : ", eval_acc)
    print("Loss : ", eval_loss)

Epoch 1/10
67/67 [==============================] - 112s 2s/step - loss: 0.7216 - accuracy: 0.5930 - val_loss: 0.5954 - val_accuracy: 0.6860
Epoch 2/10
67/67 [==============================] - 109s 2s/step - loss: 0.5784 - accuracy: 0.7075 - val_loss: 0.4413 - val_accuracy: 0.7954
Epoch 3/10
67/67 [==============================] - 109s 2s/step - loss: 0.5057 - accuracy: 0.7465 - val_loss: 0.4349 - val_accuracy: 0.7976
Epoch 4/10
67/67 [==============================] - 109s 2s/step - loss: 0.4096 - accuracy: 0.8108 - val_loss: 0.6168 - val_accuracy: 0.7702
Epoch 5/10
67/67 [==============================] - 109s 2s/step - loss: 0.3396 - accuracy: 0.8521 - val_loss: 0.3542 - val_accuracy: 0.8479
Epoch 6/10
67/67 [==============================] - 107s 2s/step - loss: 0.2911 - accuracy: 0.8709 - val_loss: 0.2852 - val_accuracy: 0.8851
Epoch 7/10
67/67 [==============================] - 107s 2s/step - loss: 0.2575 - accuracy: 0.8930 - val_loss: 0.3025 - val_accuracy: 0.8556
Epoch 8/10
67

In [4]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_1.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_1.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#para parar el entrenamiento cuando llega a 90
#early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del y_val
del X_train
del y_train

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
67/67 [==============================] - 92s 1s/step - loss: 0.7293 - accuracy: 0.5981 - val_loss: 0.5933 - val_accuracy: 0.5930
Epoch 2/10
67/67 [==============================] - 90s 1s/step - loss: 0.5897 - accuracy: 0.5981 - val_loss: 0.5433 - val_accuracy: 0.5930
Epoch 3/10
67/67 [==============================] - 90s 1s/step - loss: 0.5377 - accuracy: 0.5981 - val_loss: 0.5412 - val_accuracy: 0.5930
Epoch 4/10
67/67 [==============================] - 90s 1s/step - loss: 0.4352 - accuracy: 0.5981 - val_loss: 0.3728 - val_accuracy: 0.5930
Epoch 5/10
67/67 [==============================] - 91s 1s/step - loss: 0.3762 - accuracy: 0.5981 - val_loss: 0.3271 - val_accuracy: 0.5930
Epoch 6/10
67/67 [==============================] - 91s 1s/step - loss: 0.3453 - accuracy: 0.5981 - val_loss: 0.2976 - val_accuracy: 0.5930
Epoch 7/10
67/67 [==============================] - 91s 1s/step - loss: 0.2641 - accuracy: 0.5981 - val_loss: 0.2667 - val_accuracy: 0.5930
Epoch 8/10
67/67 [==

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_2.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_2.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#para parar el entrenamiento cuando llega a 90
#early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del y_val
del X_train
del y_train

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
69/69 [==============================] - 95s 1s/step - loss: 0.7541 - accuracy: 0.5739 - val_loss: 0.6813 - val_accuracy: 0.5610
Epoch 2/10
69/69 [==============================] - 94s 1s/step - loss: 0.6744 - accuracy: 0.5739 - val_loss: 0.6593 - val_accuracy: 0.5610
Epoch 3/10
69/69 [==============================] - 94s 1s/step - loss: 0.6401 - accuracy: 0.5739 - val_loss: 0.6028 - val_accuracy: 0.5610
Epoch 4/10
69/69 [==============================] - 94s 1s/step - loss: 0.5012 - accuracy: 0.5739 - val_loss: 0.3825 - val_accuracy: 0.5610
Epoch 5/10
69/69 [==============================] - 94s 1s/step - loss: 0.4186 - accuracy: 0.5739 - val_loss: 0.3541 - val_accuracy: 0.5610
Epoch 6/10
69/69 [==============================] - 94s 1s/step - loss: 0.3299 - accuracy: 0.5739 - val_loss: 0.3243 - val_accuracy: 0.5610
Epoch 7/10
69/69 [==============================] - 94s 1s/step - loss: 0.2814 - accuracy: 0.5739 - val_loss: 0.2928 - val_accuracy: 0.5610
Epoch 8/10
69/69 [==

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_3.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_3.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#para parar el entrenamiento cuando llega a 90
#early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del y_val
del X_train
del y_train

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\4.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_4.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#para parar el entrenamiento cuando llega a 90
#early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del y_val
del X_train
del y_train

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_5.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_5.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#para parar el entrenamiento cuando llega a 90
#early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del y_val
del X_train
del y_train

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
42/67 [=================>............] - ETA: 28s - loss: 0.6025 - accuracy: 0.7783

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_6.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_6.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#para parar el entrenamiento cuando llega a 90
#early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del y_val
del X_train
del y_train

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_7.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_7.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#para parar el entrenamiento cuando llega a 90
#early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del y_val
del X_train
del y_train

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_8.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_8.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#para parar el entrenamiento cuando llega a 90
#early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del y_val
del X_train
del y_train

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

In [21]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_9.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_9.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#para parar el entrenamiento cuando llega a 90
#early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del y_val
del X_train
del y_train

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

Epoch 1/10
33/33 [==============================] - 54s 2s/step - loss: 0.0508 - accuracy: 0.9847 - val_loss: 0.0049 - val_accuracy: 1.0000
Epoch 2/10
33/33 [==============================] - 54s 2s/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 3/10
33/33 [==============================] - 54s 2s/step - loss: 2.3941e-04 - accuracy: 1.0000 - val_loss: 7.1710e-04 - val_accuracy: 1.0000
Epoch 4/10
33/33 [==============================] - 54s 2s/step - loss: 2.1873e-04 - accuracy: 1.0000 - val_loss: 3.3291e-04 - val_accuracy: 1.0000
Epoch 5/10
33/33 [==============================] - 54s 2s/step - loss: 9.4058e-04 - accuracy: 1.0000 - val_loss: 3.0015e-04 - val_accuracy: 1.0000
Epoch 6/10
33/33 [==============================] - 54s 2s/step - loss: 4.9740e-04 - accuracy: 1.0000 - val_loss: 2.7708e-04 - val_accuracy: 1.0000
Epoch 7/10
33/33 [==============================] - 54s 2s/step - loss: 6.2706e-05 - accuracy: 1.0000 - val_loss: 2.4691e-04 - v

In [ ]:
X = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\X_10.npy')
y = np.load(f'C:\\Users\\47575909\\Desktop\\Dataset\\y_10.npy')

#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#para parar el entrenamiento cuando llega a 90
#early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, mode='max', verbose=1, baseline=0.9)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)
    
del X
del y
del X_val
del y_val
del X_train
del y_train

gc.collect()

print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss)

In [ ]:
#ABAJO VOY A TESTEAR CON LOS DATOS QUE NO USÉ

In [4]:
modelo_entrenado = tf.keras.models.load_model(f'C:\\Users\\47575909\\Desktop\\modelo_SinapsIA_SIGMOID_79.h5')

In [12]:
modelo_entrenado.predict_classes

AttributeError: 'Functional' object has no attribute 'predict_classes'

In [8]:
prueba_normal = np.load('C:\\Users\\47575909\\Desktop\\X_predict_epilepsy.npy')

#HAGO UNA PREDICCIÓN
prediction = modelo_entrenado.predict(prueba_normal)

epilepsy_prob = np.mean(prediction)

print("Probabilidad de tener epilepsia:", epilepsy_prob)

print(prediction)

10/10 [==============================] - 10s 1s/step
Probabilidad de tener epilepsia: 0.4249218
[[0.9999948 ]
 [0.99993026]
 [0.9999456 ]
 [0.03416435]
 [0.05418378]
 [0.31896675]
 [0.5249109 ]
 [0.3246847 ]
 [0.31333873]
 [0.33310542]
 [0.5062641 ]
 [0.8427809 ]
 [0.35349497]
 [0.5952718 ]
 [0.8391339 ]
 [0.59773815]
 [0.2144472 ]
 [0.4976822 ]
 [0.28451973]
 [0.4789209 ]
 [0.401643  ]
 [0.5452272 ]
 [0.624994  ]
 [0.0762439 ]
 [0.11877525]
 [0.2138696 ]
 [0.5447712 ]
 [0.76375747]
 [0.7653331 ]
 [0.60560906]
 [0.5612265 ]
 [0.45235023]
 [0.52263856]
 [0.5505097 ]
 [0.75837237]
 [0.8001266 ]
 [0.20156442]
 [0.15673877]
 [0.786269  ]
 [0.280881  ]
 [0.3535062 ]
 [0.49214447]
 [0.2864968 ]
 [0.874659  ]
 [0.9090262 ]
 [0.5515237 ]
 [0.57678795]
 [0.6922687 ]
 [0.52214473]
 [0.28233144]
 [0.13427973]
 [0.7176633 ]
 [0.82249916]
 [0.84875685]
 [0.11658891]
 [0.08869467]
 [0.17972536]
 [0.7178708 ]
 [0.50567913]
 [0.28270242]
 [0.40568545]
 [0.47764954]
 [0.3574375 ]
 [0.7140959 ]
 [0.3117

In [19]:
modelo_entrenado.layers[-1].activation

<function keras.src.activations.softmax(x, axis=-1)>

In [6]:
half_prueba = np.load('C:\\Users\\47575909\\Desktop\\X_predict_normal.npy')
#HAGO UNA PREDICCIÓN
prediction = modelo_entrenado.predict(half_prueba)

#probability = np.mean(prediction)
#epilepsy_prob = prediction[:, 1]
#print("Probabilidad de tener epilepsia:", epilepsy_prob)

print(prediction)

10/10 [==============================] - 10s 993ms/step
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [

In [3]:
from keras.utils import plot_model
plot_model(modelo_entrenado, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [5]:
# Cargar los archivos X e y
X = np.load('C:\\Users\\47575909\\Desktop\\Dataset\\X_4.npy')
y = np.load('C:\\Users\\47575909\\Desktop\\Dataset\\y_4.npy')

# Especificar el número de partes en las que deseas dividir los archivos
num_partes = 4

# Calcular el tamaño de cada parte
tamaño_parte_X = len(X) // num_partes
tamaño_parte_y = len(y) // num_partes

# Dividir los archivos en partes
partes_X = []
partes_y = []
inicio = 0
for i in range(num_partes - 1):
    partes_X.append(X[inicio:inicio + tamaño_parte_X])
    partes_y.append(y[inicio:inicio + tamaño_parte_y])
    inicio += tamaño_parte_X

# Añadir la última parte (puede que tenga un tamaño diferente si la longitud no es divisible por el número de partes)
partes_X.append(X[inicio:])
partes_y.append(y[inicio:])

for i, (parte_X, parte_y) in enumerate(zip(partes_X, partes_y)):
    model.evaluate(parte_X, parte_y)

16/74 [=====>........................] - ETA: 1:07 - loss: 3.3450 - accuracy: 0.1836

KeyboardInterrupt: 

In [7]:
'''X = np.load('C:\\Users\\47575909\\Desktop\\Dataset\\X_1.npy')
y = np.load('C:\\Users\\47575909\\Desktop\\Dataset\\y_1.npy')'''

In [ ]:
'''#DIVIDO EN TRAIN, TEST Y EVALUATION
    #Divido los datos de train de los de test y evaluation (70/30)
    X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

    #Divido los datos de test de los de evaluation (50/50 del 30% anterior)
    X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)'''

In [8]:
'''model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[wandb.keras.WandbCallback())

#EVALÚO
eval_loss, eval_acc = model.evaluate(X_val, y_val)

wandb.log({"eval_loss": test_loss, "eval_acc": test_acc})'''

Epoch 1/10
 9/96 [=>............................] - ETA: 1:39 - loss: 0.9005 - accuracy: 0.6111

KeyboardInterrupt: 

In [ ]:
'''print("Accuracy : ", eval_acc)
print("Loss : ", eval_loss'''

In [6]:
'''def model_train(archivo_x, archivo_y):
    X = np.load(archivo_x)
    y = np.load(archivo_y)

    #DIVIDO EN TRAIN, TEST Y EVALUATION
    #Divido los datos de train de los de test y evaluation (70/30)
    X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

    #Divido los datos de test de los de evaluation (50/50 del 30% anterior)
    X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

    
    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[wandb.keras.WandbCallback())

    #EVALÚO
    eval_loss, eval_acc = model.evaluate(X_val, y_val)

    wandb.log({"eval_loss": test_loss, "eval_acc": test_acc})

    #VEO LA PERFORMANCE
    print("Accuracy : ", eval_acc)
    print("Loss : ", eval_loss)'''

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (1637144083.py, line 13)

In [25]:
'''directorio = 'C:\\Users\\47575909\\Desktop\\Dataset'
num_archivos = 10

for i in range(1, num_archivos + 1):
    archivo_x = os.path.join(directorio, f'X_{i}.npy')
    archivo_y = os.path.join(directorio, f'y_{i}.npy')

    model_train(archivo_x, archivo_y)'''

ValueError: Found input variables with inconsistent numbers of samples: [8181, 2727]

In [ ]:
'''X = np.load('C:\\Users\\47575909\\Desktop\\X.npy')
y = np.load('C:\\Users\\47575909\\Desktop\\y.npy')'''

In [ ]:
'''#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)'''

In [ ]:
'''#ENTRENO
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
test_loss, test_acc = model.evaluate(X_val, y_val)'''

In [ ]:
'''#VEO LA PERFORMANCE
print("Accuracy : ", test_acc)
print("Loss : ", test_loss)'''

In [27]:
prueba_normal = np.load('C:\\Users\\47575909\\Desktop\\X_predict_epilepsy.npy')

#HAGO UNA PREDICCIÓN
prediction = model.predict(prueba_normal)

probability = np.mean(prediction)
#epilepsy_prob = prediction[:, 1]
#print("Probabilidad de tener epilepsia:", epilepsy_prob)

print(probability)

10/10 [==============================] - 10s 1s/step
6.024863e-08


##### Decirle a chona que el indio usa ResNet-26D y que keras no lo tiene ese. Que hago?...
- Uso otro modelo de ResNet
- Uso las liberías pytorch y timm que si tienen el ResNet-26D

# Exportamos el modelo
(Con tensorflow.js)

In [ ]:
#guardamos el modelo
model.save('modelo_SinapsIA_SIGMOID_3.h5')

In [ ]:
'''#Exportamos el modelooo
tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --signature_name=serving_default --saved_model_tags=serve /ruta/al/modelo/saved_model /ruta/de/destino/
#che, no entiendo lo de las rutas'''

In [ ]:
//HABRÍA QUE HACER ESTO PARA QUE ME DEVUELVA EN LA WEB SOLO LA PROBABILIDAD QUE YO QUIERO (la probabilidad de tener epilepsia)

// Obtener datos de entrada (pueden ser simulados en este ejemplo)
const datosEntrada = obtenerDatosDeEntrada();

// Cargar el modelo
const modelo = await cargarModelo();

// Realizar predicciones
const predicciones = modelo.predict(tf.tensor(datosEntrada));

// Mostrar la probabilidad de tener epilepsia
const probabilidadEpilepsia = predicciones.mean();
console.log('Probabilidad de tener epilepsia:', probabilidadEpilepsia);